အောက်ပါ notebook သည် GitHub Copilot Chat မှ အလိုအလျောက်ဖန်တီးထားပြီး မူလတပ်ဆင်ခြင်းအတွက်သာ ရည်ရွယ်ပါသည်


# Prompt Engineering အကြောင်းအရာမိတ်ဆက်

Prompt engineering ဆိုတာ သဘာဝဘာသာစကားလုပ်ငန်းစဉ်တွေမှာ အသုံးပြုမယ့် prompt တွေကို ဒီဇိုင်းဆွဲခြင်းနဲ့ အကောင်းဆုံးဖြစ်အောင် ပြင်ဆင်ခြင်းလုပ်ငန်းစဉ်ပါ။ ဒီလုပ်ငန်းစဉ်မှာ မှန်ကန်တဲ့ prompt ကို ရွေးချယ်ခြင်း၊ parameter တွေကို ချိန်ညှိခြင်းနဲ့ အလုပ်လုပ်ဆောင်မှုကို တိုင်းတာသုံးသပ်ခြင်းတွေ ပါဝင်ပါတယ်။ Prompt engineering က NLP မော်ဒယ်တွေမှာ တိကျမှုမြင့်မားပြီး ထိရောက်မှုရှိစေဖို့ အရေးကြီးပါတယ်။ ဒီအပိုင်းမှာတော့ OpenAI မော်ဒယ်တွေကို အသုံးပြုပြီး prompt engineering အခြေခံအကြောင်းအရာတွေကို လေ့လာသွားမှာပါ။


### လေ့ကျင့်ခန်း ၁: စကားလုံးခွဲခြမ်းခြင်း
OpenAI မှ ထုတ်လုပ်ထားတဲ့ အခမဲ့၊ မြန်မြန်ဆန်ဆန် စကားလုံးခွဲခြမ်းစက် tiktoken ကို အသုံးပြုပြီး စကားလုံးခွဲခြမ်းခြင်းကို လေ့လာပါ။
နောက်ထပ် ဥပမာများအတွက် [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) ကို ကြည့်ပါ။


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### လေ့ကျင့်ခန်း ၂: OpenAI API Key တပ်ဆင်မှုကို အတည်ပြုပါ

အောက်ပါကုဒ်ကို ပြေးဆွဲပြီး သင့် OpenAI endpoint ကို မှန်ကန်စွာ တပ်ဆင်ထားသည်ကို စစ်ဆေးပါ။ ဒီကုဒ်က အခြေခံ prompt တစ်ခုကိုသာ စမ်းသပ်ပြီး completion ကို အတည်ပြုပါမယ်။ `oh say can you see` ဆိုတဲ့ input ကို ထည့်လိုက်ရင် `by the dawn's early light..` ဆိုတဲ့အတိုင်း ဆက်ပြီး ပြည့်စုံလာရပါမယ်။


In [ ]:
# The OpenAI SDK was updated on Nov 8, 2023 with new guidance for migration
# See: https://github.com/openai/openai-python/discussions/742

## Updated
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

## Updated
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]       
    response = client.chat.completions.create(   
        model=deployment,                                         
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens=1024
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt)
print(response)

### လေ့ကျင့်ခန်း ၃ - ဖန်တီးမှုများ

မရှိနိုင်တဲ့အကြောင်းအရာတစ်ခု၊ ဒါမှမဟုတ် LLM ရဲ့ ကြိုတင်လေ့ကျင့်ထားတဲ့ဒေတာအတွင်း မပါဝင်တဲ့ (နောက်ဆုံးပေါ်) အကြောင်းအရာတစ်ခုအကြောင်းနဲ့ ပတ်သက်ပြီး prompt တစ်ခုကို ဖြေဆိုပေးဖို့ LLM ကို တောင်းဆိုလိုက်တဲ့အခါ ဘာတွေဖြစ်လာနိုင်သလဲဆိုတာ စမ်းသပ်ကြည့်ပါ။ တစ်ခြား prompt တစ်ခု သို့မဟုတ် တစ်ခြားမော်ဒယ်တစ်ခုကို အသုံးပြုပြီး စမ်းကြည့်တဲ့အခါ ဖြေကြားချက်ဘယ်လိုပြောင်းလဲသွားတယ်ဆိုတာလည်း ကြည့်ပါ။


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt)
print(response)

### လေ့ကျင့်ခန်း ၄: ညွှန်ကြားချက်အခြေခံ

"text" ဆိုတဲ့ variable ကို အဓိကအကြောင်းအရာအဖြစ် သတ်မှတ်ပါ  
"prompt" ဆိုတဲ့ variable ကို အဓိကအကြောင်းအရာနဲ့ ဆက်စပ်တဲ့ ညွှန်ကြားချက်တစ်ခု ထည့်ပေးပါ။

ဒီမှာတော့ model ကို အဓိကအကြောင်းအရာကို ဒုတိယတန်းကျောင်းသားအတွက် အကျဉ်းချုပ်ပေးပါလို့ တောင်းဆိုထားပါတယ်။


In [ ]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt)
print(response)

### လေ့ကျင့်ခန်း ၅ - ရှုပ်ထွေးသော Prompt 
System, user နဲ့ assistant messages တွေပါဝင်တဲ့ တောင်းဆိုမှုတစ်ခုကို စမ်းကြည့်ပါ 
System က assistant ရဲ့ context ကို သတ်မှတ်ပေးပါတယ်
User နဲ့ Assistant messages တွေက စကားပြောအကြောင်းအရာကို အကြိမ်ကြိမ်ပြန်လည်ပေးပို့ပေးပါတယ်

System context ထဲမှာ assistant ရဲ့ ကိုယ်ရည်ကိုယ်သွေးကို "စော်ကားစကားပြောတတ်တဲ့" ပုံစံနဲ့ သတ်မှတ်ထားတာကို သတိထားကြည့်ပါ။ 
အခြားသော ကိုယ်ရည်ကိုယ်သွေး context တစ်ခုကိုလည်း စမ်းသုံးကြည့်နိုင်ပါတယ်။ ဒါမှမဟုတ် input/output messages အစီအစဉ်အခြားတစ်ခုကိုလည်း စမ်းကြည့်နိုင်ပါတယ်


In [ ]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

### လေ့ကျင့်ခန်း - သင့်အတွေးအမြင်ကို စူးစမ်းကြည့်ပါ
အထက်ပါ ဥပမာတွေက သင့်အနေနဲ့ အသစ်သစ်သော prompt တွေ (ရိုးရှင်းတဲ့ prompt, ရှုပ်ထွေးတဲ့ prompt, ညွှန်ကြားချက်ပါဝင်တဲ့ prompt စသဖြင့်) ဖန်တီးနိုင်ဖို့ အသုံးပြုနိုင်တဲ့ ပုံစံတွေကို ပေးထားပါတယ်။ ဥပမာတွေ၊ အကြံပြုချက်တွေ၊ အခြားစိတ်ဝင်စားစရာအကြောင်းအရာတွေလိုမျိုး ကျွန်တော်တို့ ပြောခဲ့တဲ့ အကြောင်းအရာတွေကို ပိုမိုလေ့လာနိုင်ဖို့ အခြားသော လေ့ကျင့်ခန်းတွေကို ကိုယ်တိုင် ဖန်တီးကြည့်ပါ။



---

**သတိပြုရန်**:
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်ချက်များတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတရားရှိသော ရင်းမြစ်အဖြစ် ယူဆသင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များ၏ ဝန်ဆောင်မှုကို အသုံးပြုရန် အကြံပြုပါသည်။ ဤဘာသာပြန်ချက်ကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော နားလည်မှုမှားခြင်း သို့မဟုတ် အနားလည်အမှားများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
